In [1]:
import sys
sys.path.append('/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/')
from trained_untrained_results_funcs import find_best_layer,load_mean_sem_perf, loop_through_datasets
import numpy as np
from matplotlib import pyplot as plt

In [2]:
gpt2_xl = np.load('/data/LLMs/data_processed/blank/acts/X_gpt2-xl.npz')
data_labels = np.load('/data/LLMs/data_processed/blank/dataset/data_labels_blank.npy')


In [3]:
def shift_model_activations(model_acts, story_labels):

    samples = model_acts.shape[0]
    features = model_acts.shape[1]

    shifted_versions = np.zeros((samples, features*4))
    
    zero_pad = np.zeros(features)
        
    stories = np.unique(story_labels)
    for story in stories:
        story_idxs = np.argwhere(story_labels==story)
            
        model_acts_story = model_acts[story_idxs].squeeze()
        model_acts_story_2 = np.vstack((model_acts_story[1:, :], zero_pad)) # 2 second lag
        model_acts_story_6 = np.vstack((zero_pad, model_acts_story[:-1, :])) # 6 second lag
        model_acts_story_8 = np.vstack((zero_pad, zero_pad, model_acts_story[:-2, :])) # 8 second lag
        
        model_acts_time_lagged_story = np.hstack((model_acts_story_2, model_acts_story, model_acts_story_6, model_acts_story_8))
        
        shifted_versions[story_idxs.squeeze()] = model_acts_time_lagged_story
        
    return shifted_versions
    

In [4]:
gpt2_xl_stacked = {}
for layer_name, layer in gpt2_xl.items():
    
    gpt2_xl_stacked[layer_name] = shift_model_activations(layer, data_labels)
    

In [5]:
np.savez("/data/LLMs/data_processed/blank/acts/X_timelagged-gpt2-xl.npz", **gpt2_xl_stacked)

In [10]:
subjects_storage = {}
feature_extraction = ['-mp', '-sp', '']
metric = ['pearson_r', 'out_of_sample_r2']
for fe in feature_extraction:
    for m in metric:
        subjects_storage['blank'] = np.load(f"/data/LLMs/data_processed/blank/dataset/subjects.npy", allow_pickle=True)

        layer_perf_dict, best_layer_blank, layer_perf_best, layer_perf_best_across_subject  = find_best_layer(np.arange(49), '', '', resultsPath='/data/LLMs/brainscore/results_blank/', 
                            subjects=subjects_storage['blank'],
                            dataset='blank', perf=m, 
                            selected_network_indices = None, feature_extraction =fe, model_name='timelagged-gpt2-xl', seed_number=None, 
                            return_SE=False, niter=1)
        
        print(fe, m)
        print(layer_perf_dict[best_layer_blank])
        print(layer_perf_best_across_subject)

-mp pearson_r
0.04587013265400093
-mp out_of_sample_r2
0.001949681925983652
-sp pearson_r
0.041225438850380935
-sp out_of_sample_r2
0.0011908467721330788
 pearson_r
0.031958195735659514
 out_of_sample_r2
0.00011144144901155857


In [12]:
print(layer_perf_best.shape)

(60,)


In [46]:
layer_perf_dict

{0: 0.035582324570484755,
 1: 0.03768965673582303,
 2: 0.041607410569049506,
 3: 0.04587013265400093,
 4: 0.04224277698241717,
 5: 0.03835332020773615,
 6: 0.03300747642090639,
 7: 0.02969561252318048,
 8: 0.028682187144289762,
 9: 0.027176770057543688,
 10: 0.027271272225770415,
 11: 0.02745013733654747,
 12: 0.02640949966268622,
 13: 0.026381127830513235,
 14: 0.02584866615542982,
 15: 0.025444682791340816,
 16: 0.025028847972939323,
 17: 0.02568691921229572,
 18: 0.025349834474299716,
 19: 0.024112924863530852,
 20: 0.024747528286099253,
 21: 0.023779697788066975,
 22: 0.02302143091268056,
 23: 0.023152685618536787,
 24: 0.023821807418134777,
 25: 0.02461639168983823,
 26: 0.023396391804153892,
 27: 0.021804243322758085,
 28: 0.02199302536270511,
 29: 0.021094771497439723,
 30: 0.02149822604413648,
 31: 0.021255935169157678,
 32: 0.020325554336762072,
 33: 0.020846279423530435,
 34: 0.021120408907636418,
 35: 0.02205776526712663,
 36: 0.022149164366517312,
 37: 0.019921141043504505,